# Some Imports

In [1]:
import warnings
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.base import BaseEstimator, ClassifierMixin

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# Part I : Select a dataset

In [2]:
# target_url = ("https://archive.ics.uci.edu/dataset/545/rice+cammeo+and+osmancik")
target_url = 'docs/Rice_Cammeo_Osmancik.csv' # if you work local dataset, faster than a link
rice_df = pd.read_csv(target_url)

In [3]:
X, y = rice_df.drop(columns='Class'), rice_df['Class']
y = y.map({'Cammeo': 0, 'Osmancik': 1}) # encode labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Split

print(X_train.shape, X_test.shape)

(3048, 7) (762, 7)


In [4]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Part II: Train a multi-layer perceptron

In [5]:
# Only MLP
mlp_base = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
mlp_base.fit(X_train, y_train)
y_pred_mlp = mlp_base.predict(X_test)

print("Only MLP Classifier Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("Classification Report for Only MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

#AdaBoost With MLP estimator
ada_boost = BaggingClassifier(base_estimator=mlp_base, n_estimators=50, random_state=42)
ada_boost.fit(X_train, y_train)
y_pred_ada = ada_boost.predict(X_test)

print("AdaBoost with MLP Classifier Accuracy:", accuracy_score(y_test, y_pred_ada))
print("Classification Report for AdaBoost with MLP Classifier:\n", classification_report(y_test, y_pred_ada))

Only MLP Classifier Accuracy: 0.9291338582677166
Classification Report for Only MLP Classifier:
               precision    recall  f1-score   support

           0       0.93      0.92      0.92       350
           1       0.93      0.94      0.93       412

    accuracy                           0.93       762
   macro avg       0.93      0.93      0.93       762
weighted avg       0.93      0.93      0.93       762

AdaBoost with MLP Classifier Accuracy: 0.926509186351706
Classification Report for AdaBoost with MLP Classifier:
               precision    recall  f1-score   support

           0       0.92      0.92      0.92       350
           1       0.93      0.93      0.93       412

    accuracy                           0.93       762
   macro avg       0.93      0.93      0.93       762
weighted avg       0.93      0.93      0.93       762



## Part III: Train a random decision forest with perceptron

In [6]:
# Custom decision tree with each node uses a Perceptron
class PerceptronDecisionTree(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth=3, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree_ = None

    def fit(self, X, y):
        self.tree_ = self._fit(X, y, depth=0)
        return self

    def _fit(self, X, y, depth):
        if len(set(y)) == 1 or len(y) < self.min_samples_split or depth == self.max_depth:
            return np.argmax(np.bincount(y))

        perceptron = Perceptron()
        perceptron.fit(X, y)

        y_pred = perceptron.predict(X)
        left_mask = y_pred == 0
        right_mask = ~left_mask

        if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
            return np.argmax(np.bincount(y))

        node = {
            'perceptron': perceptron,
            'left': self._fit(X[left_mask], y[left_mask], depth + 1),
            'right': self._fit(X[right_mask], y[right_mask], depth + 1)
        }
        return node

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree_) for x in X])

    def _predict_one(self, x, node):
        if isinstance(node, dict):
            perceptron = node['perceptron']
            y_pred = perceptron.predict([x])[0]
            if y_pred == 0:
                return self._predict_one(x, node['left'])
            else:
                return self._predict_one(x, node['right'])
        else:
            return node

# Perceptron Decision Tree
perceptron_tree = PerceptronDecisionTree(max_depth=5, min_samples_split=10)
perceptron_tree.fit(X_train, y_train)
y_pred = perceptron_tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.9239
